In [4]:
def get_script_code(ipy_file_path, target_file_name="target-file.py"):
    from nbformat import read, NO_CONVERT

    with open(ipy_file_path) as fp:
        notebook = read(fp, NO_CONVERT)

    cells = notebook['cells']
    code_cells = [c for c in cells if c['cell_type'] == 'code']
    notebook_script = ""
    for cell in code_cells:
        notebook_script += cell.source
    print(notebook_script)
    with open(target_file_name, "w") as f_obj:
        f_obj.write(notebook_script)

get_script_code("tf_model.ipynb")

import numpy as np
import tensorflow as tf


def train_step(images, labels, mnist_model, optimizer, loss_object):
    loss_history = []
    with tf.GradientTape() as tape:
        logits = mnist_model(images, training=True)
        tf.debugging.assert_equal(logits.shape, (32, 10))
        loss_value = loss_object(labels, logits)
    loss_history.append(loss_value.numpy().mean())
    grads = tape.gradient(loss_value, mnist_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))
    return np.mean(loss_history), mnist_model


def train(epochs, dataset, train_step):
    tot_loss = []
    weights = None

    mnist_model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16,[3,3], activation='relu',
                         input_shape=(None, None, 1)),
        tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(10)
    ])

    optimizer = tf.keras.optimizers.Ad